In [3]:
----------------------
-- CREATE DIM TABLES--
----------------------

DROP TABLE IF EXISTS Gold.dim_countries;

CREATE TABLE Gold.dim_countries
AS 
SELECT row_number() OVER(ORDER BY all_countries.country) AS country_id, 
    all_countries.country AS country_name
FROM (
    SELECT DISTINCT country AS country FROM Silver.athletes
    UNION
    SELECT DISTINCT country AS country FROM Silver.coaches
    UNION
    SELECT DISTINCT team_country AS country FROM Silver.medals
    UNION
    SELECT DISTINCT country AS country FROM Silver.teams
) AS all_countries;


DROP TABLE IF EXISTS Gold.dim_disciplines;

CREATE TABLE Gold.dim_disciplines
AS
SELECT row_number() OVER(ORDER BY all_disciplines.discipline) AS discipline_id, 
    all_disciplines.discipline AS discipline_name
FROM (
    SELECT DISTINCT discipline FROM Silver.athletes
    UNION
    SELECT DISTINCT discipline FROM Silver.coaches
    UNION
    SELECT DISTINCT discipline FROM Silver.teams
) AS all_disciplines;


DROP TABLE IF EXISTS Gold.dim_events;

CREATE TABLE Gold.dim_events
AS
SELECT row_number() OVER(ORDER BY all_events.event) AS event_id, 
    UPPER(TRIM(all_events.event)) AS event_name
FROM (
    SELECT DISTINCT event FROM Silver.coaches
    UNION
    SELECT DISTINCT event FROM Silver.teams
) AS all_events;

StatementMeta(, abe48e0e-2cc1-41fa-9093-a7fb8d1160c3, 14, Finished, Available, Finished)

<Spark SQL result set with 0 rows and 0 fields>

<Spark SQL result set with 0 rows and 0 fields>

<Spark SQL result set with 0 rows and 0 fields>

<Spark SQL result set with 0 rows and 0 fields>

<Spark SQL result set with 0 rows and 0 fields>

<Spark SQL result set with 0 rows and 0 fields>

In [4]:
--------------------------
-- Load Dim Gold Tables --
--------------------------

TRUNCATE TABLE Gold.dim_countries;

INSERT INTO Gold.dim_countries
SELECT row_number() OVER(ORDER BY all_countries.country) AS country_id, 
    all_countries.country AS country_name
FROM (
    SELECT DISTINCT country AS country FROM Silver.athletes
    UNION
    SELECT DISTINCT country AS country FROM Silver.coaches
    UNION
    SELECT DISTINCT team_country AS country FROM Silver.medals
    UNION
    SELECT DISTINCT country AS country FROM Silver.teams
) AS all_countries;


TRUNCATE TABLE Gold.dim_disciplines;

INSERT INTO Gold.dim_disciplines
SELECT row_number() OVER(ORDER BY all_disciplines.discipline) AS discipline_id, 
    all_disciplines.discipline AS discipline_name
FROM (
    SELECT DISTINCT discipline FROM Silver.athletes
    UNION
    SELECT DISTINCT discipline FROM Silver.coaches
    UNION
    SELECT DISTINCT discipline FROM Silver.teams
) AS all_disciplines;


TRUNCATE TABLE Gold.dim_events;

INSERT INTO Gold.dim_events
SELECT row_number() OVER(ORDER BY all_events.event) AS event_id, 
    UPPER(TRIM(all_events.event)) AS event_name
FROM (
    SELECT DISTINCT event FROM Silver.coaches
    UNION
    SELECT DISTINCT event FROM Silver.teams
) AS all_events;


---------------------------
-- Load FACT Gold Tables --
---------------------------

TRUNCATE TABLE Gold.fact_medals_by_country;

INSERT INTO Gold.fact_medals_by_country
SELECT c.country_id, m.gold_medals AS gold_count, m.silver_medals AS silver_count, m.bronze_medals AS bronze_count, m.total_medals, m.rank, m.rank_by_total
FROM Silver.medals m
LEFT JOIN Gold.dim_countries c on m.team_country=c.country_name;


TRUNCATE TABLE Gold.fact_entries_gender;

INSERT INTO Gold.fact_entries_gender
SELECT d.discipline_id, eg.female AS total_female, eg.male AS total_male, eg.total
FROM Silver.entries_gender eg
LEFT JOIN Gold.dim_disciplines d on eg.discipline=d.discipline_name;


TRUNCATE TABLE Gold.fact_athletes_participation;

INSERT INTO Gold.fact_athletes_participation
SELECT a.athlete_id, c.country_id, d.discipline_id
FROM Silver.athletes a
LEFT JOIN Gold.dim_countries c on c.country_name=a.country
LEFT JOIN Gold.dim_disciplines d on d.discipline_name=a.discipline;

StatementMeta(, abe48e0e-2cc1-41fa-9093-a7fb8d1160c3, 22, Finished, , Finished)

<Spark SQL result set with 1 rows and 1 fields>

<Spark SQL result set with 0 rows and 0 fields>

<Spark SQL result set with 1 rows and 1 fields>

<Spark SQL result set with 0 rows and 0 fields>

<Spark SQL result set with 1 rows and 1 fields>

<Spark SQL result set with 0 rows and 0 fields>

<Spark SQL result set with 1 rows and 1 fields>

Error: [_LEGACY_ERROR_TEMP_DELTA_0007] A schema mismatch detected when writing to the Delta table (Table ID: ff84983e-8c0a-4343-bb0e-6ece446116b2).
To enable schema migration using DataFrameWriter or DataStreamWriter, please set:
'.option("mergeSchema", "true")'.
For other operations, set the session configuration
spark.databricks.delta.schema.autoMerge.enabled to "true". See the documentation
specific to the operation for details.

Table schema:
root
-- athlete_id: integer (nullable = true)
-- country_id: integer (nullable = true)
-- discipline_id: integer (nullable = true)


Data schema:
root
-- athlete_id: integer (nullable = true)
-- country_id: integer (nullable = true)
-- discipline_id: integer (nullable = true)
-- bronze_count: integer (nullable = true)
-- total_medals: integer (nullable = true)
-- rank: integer (nullable = true)
-- rank_by_total: integer (nullable = true)

         